In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
out_allinfo_HL = "AMC-TC-x001-HL-allinfo-mut.csv"
out_clones_HL = "AMC-TC-x001-HL-clones-mut.csv"

In [3]:
mydir = "/home/barbera/Downloads/runXXX-20210107-SC-Tom/processing/20211102-reseda/final/"
myfiles = [x for x in os.listdir(mydir) if x.endswith("-allinfo-filtered-mut.csv")]
myfiles = [mydir + x for x in myfiles]
myfiles

['/home/barbera/Downloads/runXXX-20210107-SC-Tom/processing/20211102-reseda/final/AMC-TC-x001_S0_L001.assembled-nomatch-IGH_HUMAN-allinfo-filtered-mut.csv',
 '/home/barbera/Downloads/runXXX-20210107-SC-Tom/processing/20211102-reseda/final/AMC-TC-x001_S0_L001.assembled-nomatch-IGL_HUMAN-allinfo-filtered-mut.csv',
 '/home/barbera/Downloads/runXXX-20210107-SC-Tom/processing/20211102-reseda/final/AMC-TC-x001_S0_L001.assembled-nomatch-IGK_HUMAN-allinfo-filtered-mut.csv']

In [4]:
def readFile(myfile):
    sample = myfile.split("/")[-1].split("_")[0]
    celltype = myfile.split("-")[-4]
    df = pd.read_csv(myfile)
    df["Sample"] = sample
    df["CellType"] = celltype
    df["chain"] = [x.split("_")[0] for x in df["CellType"]]
    df["CellID"] = [x.split("_")[0] for x in df["acc"]]
    return(df)

In [5]:
df = readFile(myfiles[0])
for myfile in myfiles[1:]:
    df_tmp = readFile(myfile)
    df = pd.concat([df, df_tmp])
df.tail()

,Unnamed: 0,acc,beforeMID,MID,afterMID,readingframe,cdr3pep,cdr3nuc,cdr3pepshort,cdr3nucshort,...,start.pos_y,end.pos_y,mut.count_y,mut.frac_y,align.length_y,align.seq_y,Sample,CellType,chain,CellID
3082,3082,TTTGTCAGTCCCTACT-1_contig_1,nomatch,nomatch,nomatch,0,CQQYNNWPPYTFG,TGTCAGCAGTATAATAACTGGCCTCCGTACACTTTTGGC,CQQYNN,TGTCAGCAGTATAATAAC,...,392.0,430.0,0.0,0.0,38.0,======================================,AMC-TC-x001,IGK_HUMAN,IGK,TTTGTCAGTCCCTACT-1
3083,3083,TTTGTCAGTCTAGGTT-1_contig_1,nomatch,nomatch,nomatch,0,CMQATQFPQEYTFG,TGCATGCAAGCTACACAATTTCCTCAGGAGTACACTTTTGGC,CMQATQF,TGCATGCAAGCTACACAATTT,...,395.0,433.0,0.0,0.0,38.0,======================================,AMC-TC-x001,IGK_HUMAN,IGK,TTTGTCAGTCTAGGTT-1
3084,3084,TTTGTCAGTTCGTGAT-1_contig_1,nomatch,nomatch,nomatch,2,CQQRSNWPPGITFG,TGTCAGCAGCGTAGCAACTGGCCTCCGGGAATCACTTTCGGC,CQQRSNW,TGTCAGCAGCGTAGCAACTGG,...,399.0,435.0,0.0,0.0,36.0,====================================,AMC-TC-x001,IGK_HUMAN,IGK,TTTGTCAGTTCGTGAT-1
3085,3085,TTTGTCATCAGCATGT-1_contig_1,nomatch,nomatch,nomatch,0,CQQYGSSPITFG,TGTCAGCAGTATGGTAGCTCACCGATCACCTTCGGC,CQQYG,TGTCAGCAGTATGGT,...,380.0,418.0,0.0,0.0,38.0,======================================,AMC-TC-x001,IGK_HUMAN,IGK,TTTGTCATCAGCATGT-1
3086,3086,TTTGTCATCGAGAACG-1_contig_1,nomatch,nomatch,nomatch,0,CQQYNNWPYTFG,TGTCAGCAGTATAATAACTGGCCGTACACTTTTGGC,CQQYN,TGTCAGCAGTATAAT,...,380.0,418.0,0.0,0.0,38.0,======================================,AMC-TC-x001,IGK_HUMAN,IGK,TTTGTCATCGAGAACG-1


In [6]:
# List column names
df.columns

Index(['Unnamed: 0', 'acc', 'beforeMID', 'MID', 'afterMID', 'readingframe',
       'cdr3pep', 'cdr3nuc', 'cdr3pepshort', 'cdr3nucshort', 'cdr3_qual_min',
       'cdr3_qual_max', 'cdr3_qual_avg', 'cdr3_qual', 'nt_start', 'nt_end',
       'seq_length', 'V_flag', 'V_gene', 'J_flag', 'J_gene',
       'readingframe_seq', 'seq', 'pep', 'qual', 'V_sub', 'J_sub', 'V_main',
       'acc:1', 'nr_v_mains', 'nr_v_subs', 'nr_v_alleles', 'nr_j_subs',
       'nr_j_alleles', 'acc:2', 'readingframe:1', 'nr_sites', 'gene_x',
       'cigar_x', 'start.pos_x', 'end.pos_x', 'mut.count_x', 'mut.frac_x',
       'align.length_x', 'align.seq_x', 'gene_y', 'cigar_y', 'start.pos_y',
       'end.pos_y', 'mut.count_y', 'mut.frac_y', 'align.length_y',
       'align.seq_y', 'Sample', 'CellType', 'chain', 'CellID'],
      dtype='object')

In [7]:
# Get the cell barcode
f = lambda x: x.split("-")[0]
df["barcode"] = [x for x in map(f, df["CellID"])]

In [8]:
# Split data in two dataframes, one for heavy and one for the light chain
df_heavy = df[df["CellType"] == "IGH_HUMAN"]
df_light = df[df["CellType"] != "IGH_HUMAN"]
print("Heavy:", len(df_heavy))
print("Light:", len(df_light))

Heavy: 4271
Light: 4103


In [9]:
# Create short version of the CDR3's (consistent with IMGT)
df_heavy["cdr3pep_short"] = [x[:-7] for x in df_heavy["cdr3pep"]]
df_light["cdr3pep_short"] = [x[:-1] for x in df_light["cdr3pep"]]
#df_heavy.head()

/home/barbera/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/barbera/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# add column with the annotation in the same format as cellranger does
def combine_chaininfo_cdr3 (chain, cdr3_short):
    return(chain.split("_")[0] + ":" + cdr3_short)
print(combine_chaininfo_cdr3("IGH_HUMAN", "CATHLIGSHFDYW"))

IGH:CATHLIGSHFDYW


In [11]:
df_heavy["chain_cdr3"] = [x for x in map(combine_chaininfo_cdr3, df_heavy["CellType"], df_heavy["cdr3pep_short"])]
df_heavy.head()

/home/barbera/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,acc,beforeMID,MID,afterMID,readingframe,cdr3pep,cdr3nuc,cdr3pepshort,cdr3nucshort,...,mut.frac_y,align.length_y,align.seq_y,Sample,CellType,chain,CellID,barcode,cdr3pep_short,chain_cdr3
0,0,AAACCTGAGGCTAGGT-1_contig_2,nomatch,nomatch,nomatch,0,CAKDHSSWYEWYSLAPGDYWGQGTLVT,TGTGCGAAAGATCACAGCAGCTGGTACGAATGGTATTCCCTCGCGC...,CAKDHSSWYEWYSLAPGDYW,TGTGCGAAAGATCACAGCAGCTGGTACGAATGGTATTCCCTCGCGC...,...,0.0,40.0,========================================,AMC-TC-x001,IGH_HUMAN,IGH,AAACCTGAGGCTAGGT-1,AAACCTGAGGCTAGGT,CAKDHSSWYEWYSLAPGDYW,IGH:CAKDHSSWYEWYSLAPGDYW
1,1,AAACCTGAGTGCGATG-1_contig_1,nomatch,nomatch,nomatch,1,CATHLIGSHFDYWGQGTLVT,TGTGCGACACACCTCATCGGGTCACACTTTGACTACTGGGGCCAGG...,CATHLIGSHFDYW,TGTGCGACACACCTCATCGGGTCACACTTTGACTACTGG,...,0.0,44.0,============================================,AMC-TC-x001,IGH_HUMAN,IGH,AAACCTGAGTGCGATG-1,AAACCTGAGTGCGATG,CATHLIGSHFDYW,IGH:CATHLIGSHFDYW
2,2,AAACCTGCAATCTACG-1_contig_2,nomatch,nomatch,nomatch,0,CARYFDAFDIWGQGTMVT,TGTGCGAGATACTTCGATGCTTTTGATATCTGGGGCCAAGGGACAA...,CARYFDAFDIW,TGTGCGAGATACTTCGATGCTTTTGATATCTGG,...,0.0,48.0,================================================,AMC-TC-x001,IGH_HUMAN,IGH,AAACCTGCAATCTACG-1,AAACCTGCAATCTACG,CARYFDAFDIW,IGH:CARYFDAFDIW
3,3,AAACCTGGTTCAGTAC-1_contig_2,nomatch,nomatch,nomatch,1,CARDQAIAAADLYYFDYWGQGTLVT,TGTGCGAGAGACCAAGCTATAGCAGCAGCCGACCTTTACTACTTTG...,CARDQAIAAADLYYFDYW,TGTGCGAGAGACCAAGCTATAGCAGCAGCCGACCTTTACTACTTTG...,...,0.0,47.0,===============================================,AMC-TC-x001,IGH_HUMAN,IGH,AAACCTGGTTCAGTAC-1,AAACCTGGTTCAGTAC,CARDQAIAAADLYYFDYW,IGH:CARDQAIAAADLYYFDYW
4,4,AAACCTGGTTCATGGT-1_contig_1,nomatch,nomatch,nomatch,2,CARLGVGATGVYDYWGQGTLVT,TGTGCGAGACTGGGGGTGGGAGCTACCGGAGTTTATGACTACTGGG...,CARLGVGATGVYDYW,TGTGCGAGACTGGGGGTGGGAGCTACCGGAGTTTATGACTACTGG,...,0.0,40.0,========================================,AMC-TC-x001,IGH_HUMAN,IGH,AAACCTGGTTCATGGT-1,AAACCTGGTTCATGGT,CARLGVGATGVYDYW,IGH:CARLGVGATGVYDYW


In [12]:
df_light["chain_cdr3"] = [x for x in map(combine_chaininfo_cdr3, df_light["CellType"], df_light["cdr3pep_short"])]
df_light.head()

/home/barbera/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,acc,beforeMID,MID,afterMID,readingframe,cdr3pep,cdr3nuc,cdr3pepshort,cdr3nucshort,...,mut.frac_y,align.length_y,align.seq_y,Sample,CellType,chain,CellID,barcode,cdr3pep_short,chain_cdr3
0,0,AAACCTGAGTGCGATG-1_contig_2,nomatch,nomatch,nomatch,0,CSSYTSSSTLVFG,TGCAGCTCATATACAAGCAGCAGCACTCTCGTATTCGGC,CSSYTS,TGCAGCTCATATACAAGC,...,0.0,34.0,==================================,AMC-TC-x001,IGL_HUMAN,IGL,AAACCTGAGTGCGATG-1,AAACCTGAGTGCGATG,CSSYTSSSTLVF,IGL:CSSYTSSSTLVF
1,1,AAACCTGGTTCATGGT-1_contig_2,nomatch,nomatch,nomatch,1,CQAWDSSNVVFG,TGTCAGGCGTGGGACAGCAGCAATGTGGTATTCGGC,CQAWD,TGTCAGGCGTGGGAC,...,0.0,38.0,======================================,AMC-TC-x001,IGL_HUMAN,IGL,AAACCTGGTTCATGGT-1,AAACCTGGTTCATGGT,CQAWDSSNVVF,IGL:CQAWDSSNVVF
2,2,AAACCTGTCCCGACTT-1_contig_1,nomatch,nomatch,nomatch,1,CQAWDSSLVFG,TGTCAGGCGTGGGACAGCAGTTTGGTATTCGGC,CQAW,TGTCAGGCGTGG,...,0.0,36.0,====================================,AMC-TC-x001,IGL_HUMAN,IGL,AAACCTGTCCCGACTT-1,AAACCTGTCCCGACTT,CQAWDSSLVF,IGL:CQAWDSSLVF
3,3,AAACGGGAGGATGCGT-1_contig_2,nomatch,nomatch,nomatch,1,CQVWDSSTEVFG,TGTCAGGTGTGGGACAGCAGCACTGAGGTGTTCGGC,CQVWD,TGTCAGGTGTGGGAC,...,0.0,35.0,===================================,AMC-TC-x001,IGL_HUMAN,IGL,AAACGGGAGGATGCGT-1,AAACGGGAGGATGCGT,CQVWDSSTEVF,IGL:CQVWDSSTEVF
4,4,AAACGGGCATTATCTC-1_contig_2,nomatch,nomatch,nomatch,2,CSSYTSSSTRVFG,TGCAGCTCATATACAAGCAGCAGCACCAGGGTCTTCGGA,CSSYTS,TGCAGCTCATATACAAGC,...,0.0,34.0,==================================,AMC-TC-x001,IGL_HUMAN,IGL,AAACGGGCATTATCTC-1,AAACGGGCATTATCTC,CSSYTSSSTRVF,IGL:CSSYTSSSTRVF


In [13]:
# Combine heavy and light chain info in one dataframe
df_HL_allinfo = pd.concat([df_heavy, df_light], sort=True)
df_HL_allinfo = df_HL_allinfo.sort_values(by="acc")
#df_HL_allinfo.head()

In [14]:
df_HL_allinfo.to_csv(out_allinfo_HL, index=None)
print("Wrote", out_allinfo_HL, "to disk")

Wrote AMC-TC-x001-HL-allinfo-mut.csv to disk


## Construct a clones file per cell

Put heavy and light chain info in one entry(row) per cell

In [15]:
cols = ["barcode", "chain_cdr3", "acc", "V_sub", "J_sub", "cdr3nuc", "cdr3pep", "mut.count_x", "mut.count_y", "mut.frac_x", "mut.frac_y"]
df_heavy = df_heavy[cols]
df_heavy = df_heavy.rename(columns={'chain_cdr3': 'cdr3_heavy', 'mut.count_x': 'mut.count.V.heavy', 'mut.count_y': 'mut.count.J.heavy', 'mut.frac_x': 'mut.frac.V.heavy', 'mut.frac_y': 'mut.frac.J.heavy'})
df_light = df_light[cols]
df_light = df_light.rename(columns={'chain_cdr3': 'cdr3_light', 'mut.count_x': 'mut.count.V.light', 'mut.count_y': 'mut.count.J.light', 'mut.frac_x': 'mut.frac.V.light', 'mut.frac_y': 'mut.frac.J.light'})
df_HL = pd.merge(df_heavy, df_light, how="inner", on="barcode")
df_HL.head()

,barcode,cdr3_heavy,acc_x,V_sub_x,J_sub_x,cdr3nuc_x,cdr3pep_x,mut.count.V.heavy,mut.count.J.heavy,mut.frac.V.heavy,...,cdr3_light,acc_y,V_sub_y,J_sub_y,cdr3nuc_y,cdr3pep_y,mut.count.V.light,mut.count.J.light,mut.frac.V.light,mut.frac.J.light
0,AAACCTGAGGCTAGGT,IGH:CAKDHSSWYEWYSLAPGDYW,AAACCTGAGGCTAGGT-1_contig_2,IGHV3-23,IGHJ4,TGTGCGAAAGATCACAGCAGCTGGTACGAATGGTATTCCCTCGCGC...,CAKDHSSWYEWYSLAPGDYWGQGTLVT,0,0.0,0.000000,...,IGK:CQQYGSSPLFTF,AAACCTGAGGCTAGGT-1_contig_1,IGKV3-20,IGKJ3,TGTCAGCAGTATGGTAGCTCACCCTTATTCACTTTCGGC,CQQYGSSPLFTFG,0,0.0,0.000000,0.0
1,AAACCTGAGTGCGATG,IGH:CATHLIGSHFDYW,AAACCTGAGTGCGATG-1_contig_1,IGHV1-69,IGHJ4,TGTGCGACACACCTCATCGGGTCACACTTTGACTACTGGGGCCAGG...,CATHLIGSHFDYWGQGTLVT,0,0.0,0.000000,...,IGL:CSSYTSSSTLVF,AAACCTGAGTGCGATG-1_contig_2,IGLV2-14,IGLJ3,TGCAGCTCATATACAAGCAGCAGCACTCTCGTATTCGGC,CSSYTSSSTLVFG,2,0.0,0.006734,0.0
2,AAACCTGGTTCATGGT,IGH:CARLGVGATGVYDYW,AAACCTGGTTCATGGT-1_contig_1,IGHV4-59,IGHJ4,TGTGCGAGACTGGGGGTGGGAGCTACCGGAGTTTATGACTACTGGG...,CARLGVGATGVYDYWGQGTLVT,0,0.0,0.000000,...,IGL:CQAWDSSNVVF,AAACCTGGTTCATGGT-1_contig_2,IGLV3-1,IGLJ3,TGTCAGGCGTGGGACAGCAGCAATGTGGTATTCGGC,CQAWDSSNVVFG,0,0.0,0.000000,0.0
3,AAACCTGTCCCGACTT,IGH:CARDLSWGVVANPLTGHYW,AAACCTGTCCCGACTT-1_contig_2,IGHV3-48,IGHJ4,TGTGCGAGAGATCTTAGTTGGGGGGTGGTAGCTAATCCATTAACGG...,CARDLSWGVVANPLTGHYWGQGTLVT,1,0.0,0.003378,...,IGL:CQAWDSSLVF,AAACCTGTCCCGACTT-1_contig_1,IGLV3-1,IGLJ2,TGTCAGGCGTGGGACAGCAGTTTGGTATTCGGC,CQAWDSSLVFG,0,0.0,0.000000,0.0
4,AAACGGGAGCTCAACT,IGH:CAREYSSYEGDFDLW,AAACGGGAGCTCAACT-1_contig_1,IGHV1-69,IGHJ2,TGTGCGAGAGAGTATAGCAGCTACGAAGGTGACTTCGATCTCTGGG...,CAREYSSYEGDFDLWGRGTLVT,1,0.0,0.003378,...,IGK:CMQATQFPQEYTF,AAACGGGAGCTCAACT-1_contig_2,IGKV2-24,IGKJ2,TGCATGCAAGCTACACAATTTCCTCAGGAGTACACTTTTGGC,CMQATQFPQEYTFG,0,0.0,0.000000,0.0


In [16]:
df_HL.columns

Index(['barcode', 'cdr3_heavy', 'acc_x', 'V_sub_x', 'J_sub_x', 'cdr3nuc_x',
       'cdr3pep_x', 'mut.count.V.heavy', 'mut.count.J.heavy',
       'mut.frac.V.heavy', 'mut.frac.J.heavy', 'cdr3_light', 'acc_y',
       'V_sub_y', 'J_sub_y', 'cdr3nuc_y', 'cdr3pep_y', 'mut.count.V.light',
       'mut.count.J.light', 'mut.frac.V.light', 'mut.frac.J.light'],
      dtype='object')

In [17]:
# Group by cell barcode, keep all information in the case that there are multiple assigned light and/or heavy chains
f = lambda x: ";".join(list(set(x)))
df_HL_clones = df_HL.groupby("barcode").agg({
    "cdr3_heavy": ["nunique", f],
    "acc_x": ["nunique", f],
    "V_sub_x": ["nunique", f],
    "J_sub_x": ["nunique", f],
    "cdr3nuc_x": ["nunique", f],
    "cdr3pep_x": ["nunique", f],
    "cdr3_light": ["nunique", f],
    "acc_y": ["nunique", f],
    "V_sub_y": ["nunique", f],
    "J_sub_y": ["nunique", f],
    "cdr3nuc_y": ["nunique", f],
    "cdr3pep_y": ["nunique", f],
    "mut.count.V.heavy": [np.mean, np.median],
    "mut.count.J.heavy": [np.mean, np.median],
    "mut.count.V.light": [np.mean, np.median],
    "mut.count.J.light": [np.mean, np.median],
    "mut.frac.V.heavy": [np.mean, np.median],
    "mut.frac.J.heavy": [np.mean, np.median],
    "mut.frac.V.light": [np.mean, np.median],
    "mut.frac.J.light": [np.mean, np.median]})
df_HL_clones = df_HL_clones.reset_index()

In [18]:
df_HL_clones.head(50)

barcode cdr3_heavy  \
                        nunique   
0   AAACCTGAGGCTAGGT          1   
1   AAACCTGAGTGCGATG          1   
2   AAACCTGGTTCATGGT          1   
3   AAACCTGTCCCGACTT          1   
4   AAACGGGAGCTCAACT          1   
5   AAACGGGAGGATGCGT          1   
6   AAACGGGAGTGTCTCA          1   
7   AAACGGGCACAGACAG          1   
8   AAACGGGCATTATCTC          1   
9   AAACGGGGTCTTCAAG          1   
10  AAACGGGTCGTGGACC          1   
11  AAAGATGAGACTCGGA          1   
12  AAAGATGAGTCCTCCT          1   
13  AAAGATGCAACAACCT          1   
14  AAAGATGCACAGCGTC          1   
15  AAAGATGCATCCAACA          1   
16  AAAGATGGTAGCTAAA          1   
17  AAAGATGGTCCAACTA          1   
18  AAAGATGGTGATAAAC          1   
19  AAAGATGGTGGTCTCG          1   
20  AAAGATGTCCTCGCAT          1   
21  AAAGCAAAGAGTTGGC          1   
22  AAAGCAAGTGGACGAT          1   
23  AAAGCAAGTTATCCGA          2   
24  AAAGCAATCGATGAGG          1   
25  AAAGCAATCGGCGGTT          1   
26  AAAGCAATCGTACCGG          1   
27  AAAGCAATCTCACATT          2   
28  AAAGTAGAGAGGTAGA          1   
29  AAAGTAGAGCCAGTAG          1   
30  AAAGTAGAGCTGAAAT          1   
31  AAAGTAGAGTTGCAGG          1   
32  AAAGTAGCAGCGTTCG          1   
33  AAAGTAGCATCCTAGA          2   
34  AAAGTAGTCCCGGATG          1   
35  AAATGCCCAGACGCAA          1   
36  AAATGCCGTACAGCAG          1   
37  AAATGCCGTCTTCTCG          2   
38  AACACGTAGACGACGT          1   
39  AACACGTAGCATGGCA          1   
40  AACACGTAGCTCCCAG          1   
41  AACACGTCACCTATCC          1   
42  AACACGTCAGACAAAT          1   
43  AACACGTGTGTGACGA          1   
44  AACACGTGTTACCGAT          1   
45  AACACGTTCAGGTAAA          1   
46  AACACGTTCGATGAGG          1   
47  AACCATGAGAGACGAA          1   
48  AACCATGAGCAGGCTA          1   
49  AACCATGAGGCTACGA          1   

                                                        acc_x  \
                                             <lambda> nunique   
0                            IGH:CAKDHSSWYEWYSLAPGDYW       1   
1                                   IGH:CATHLIGSHFDYW       1   
2                                 IGH:CARLGVGATGVYDYW       1   
3                             IGH:CARDLSWGVVANPLTGHYW       1   
4                                 IGH:CAREYSSYEGDFDLW       1   
5                                    IGH:CARGQWSYSDYW       1   
6                          IGH:CATVRITMVRGVIIYDRTFDYW       1   
7                               IGH:CARDRTQQLASNWFDPW       1   
8                              IGH:CARATAPYYYDSSGYLPW       1   
9                       IGH:CARDQGEELRFLEWLPYYYYGMDVW       1   
10                       IGH:CARDVGGLLWFGESSSYYYYMDVW       1   
11                        IGH:CATVSPSYCSSTSCYTISWFDPW       1   
12           IGH:CARDSNPTLPSTRPTYYDFWSGYLGEYYYYYGMDVW       1   
13                                IGH:CAKDPSGPFNGWEYW       1   
14                            IGH:CAKVIGGAVLWFGETFDIW       1   
15                           IGH:CARDYYDSSGYYLHWYFDLW       1   
16                              IGH:CARDPRWGYTSSGFDSW       1   
17                                  IGH:CARPSQNWGLDYW       1   
18                       IGH:CARASVGYDILTGYYYYYYGMDVW       1   
19                      IGH:CARDGGVCSSTSCYWDYYYYGMDVW       1   
20                              IGH:CAKDVYGRRLRGWFDPW       1   
21                 IGH:CARHYSRPYETLIMITFGGVIAREREADYW       1   
22                              IGH:CARDWDSSSWTNWFDPW       1   
23       IGH:CARAGGNWNYHGGAHNYFDPW;IGH:CARMGVGTGAFDIW       2   
24                           IGH:CAKSPATGQTYYYYYYMDVW       1   
25                             IGH:CARKLGDQLWLKDGMDVW       1   
26                           IGH:CAREHGGPWPAVYYYGMDVW       1   
27             IGH:CARRVLNGFGFDYW;IGH:CAAAVGMQYYGMDVW       2   
28                                  IGH:CAKGKWDDAFEIW       1   
29                                  IGH:CARGPGGYYSDYW       1   
30                               IGH:CARDGSVEATGDLGYW       1   
31                       IGH:CARVPYYDFWSGYFPYYY

In [19]:
df_HL_clones = df_HL_clones.rename(columns={'<lambda>': 'concat'})
df_HL_clones.columns = ["_".join(x) for x in df_HL_clones.columns] # flatten column levels
df_HL_clones = df_HL_clones.rename(columns={'barcode_': 'barcode'})
df_HL_clones.head()

,barcode,cdr3_heavy_nunique,cdr3_heavy_concat,acc_x_nunique,acc_x_concat,V_sub_x_nunique,V_sub_x_concat,J_sub_x_nunique,J_sub_x_concat,cdr3nuc_x_nunique,...,mut.count.J.light_mean,mut.count.J.light_median,mut.frac.V.heavy_mean,mut.frac.V.heavy_median,mut.frac.J.heavy_mean,mut.frac.J.heavy_median,mut.frac.V.light_mean,mut.frac.V.light_median,mut.frac.J.light_mean,mut.frac.J.light_median
0,AAACCTGAGGCTAGGT,1,IGH:CAKDHSSWYEWYSLAPGDYW,1,AAACCTGAGGCTAGGT-1_contig_2,1,IGHV3-23,1,IGHJ4,1,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
1,AAACCTGAGTGCGATG,1,IGH:CATHLIGSHFDYW,1,AAACCTGAGTGCGATG-1_contig_1,1,IGHV1-69,1,IGHJ4,1,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.006734,0.006734,0.0,0.0
2,AAACCTGGTTCATGGT,1,IGH:CARLGVGATGVYDYW,1,AAACCTGGTTCATGGT-1_contig_1,1,IGHV4-59,1,IGHJ4,1,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
3,AAACCTGTCCCGACTT,1,IGH:CARDLSWGVVANPLTGHYW,1,AAACCTGTCCCGACTT-1_contig_2,1,IGHV3-48,1,IGHJ4,1,...,0.0,0.0,0.003378,0.003378,0.0,0.0,0.000000,0.000000,0.0,0.0
4,AAACGGGAGCTCAACT,1,IGH:CAREYSSYEGDFDLW,1,AAACGGGAGCTCAACT-1_contig_1,1,IGHV1-69,1,IGHJ2,1,...,0.0,0.0,0.003378,0.003378,0.0,0.0,0.000000,0.000000,0.0,0.0


In [20]:
print("Cells all:", len(df_HL_clones))
print("Cells with 1 heavy and 1 light chain:", len(df_HL_clones[(df_HL_clones['cdr3_heavy_nunique'] == 1) & (df_HL_clones['cdr3_light_nunique'] == 1)]))

Cells all: 2855
Cells with 1 heavy and 1 light chain: 2424


In [21]:
df_HL_clones.to_csv(out_clones_HL, index=None)
print("Wrote", out_clones_HL, "to disk")

Wrote AMC-TC-x001-HL-clones-mut.csv to disk
